In [1]:
import json

In [2]:
import random

In [11]:
class Player:
    """
    Contains three methods
        1. setup_ships: takes no parameters and returns a list of lists showing the start position
                        of showing the start positions of your ships
                        Block of row i and column j, ship label 1-5 & 0 means no ship

        2. take_turn:   returns
                        -- a list of (row, column) tuples describing your shots
                        -- eg: [(1, 1), (1, 3), (1, 5), (2, 1) (4, 5)]
                        -- Note: should be equal to the number of ships present (check function)
                        -- or a tuple that has a ship number as the first element and the direction
                        -- of its moving value
                        -- 0: up, 1: right, 2: down, 3: left  (dict)

                        TODO: Do not take more shots than the number of ships
                        TODO: Do not move the ships outside the grid or radioactive zone

        3. get_name:   returns a string that is the name of the Player

    """
    import random
    from collections import defaultdict
    
    def __init__(self):
        
        self.player_battleground = [[0 for i in range(10)] for j in range(10)]
        self.opponent_battleground = [[0 for i in range(10)] for j in range(10)]
        
        self.ships_dd = Player.defaultdict(int)
        self.status = []
        
        self.potential_list = []
        for i in range(10):
            for j in range (10):
                self.potential_list.append((i,j))
                
        # number of turns
        self.counter = 0
        

    def setup_ships(self):
        '''

        :return: a 10x10 grid with the location of our ships
        '''
        grid = [[0 for i in range(10)] for j in range(10)]
        
        # ship numbers
        ship = [i for i in range(1,6)]

        # randomly shuffle the ship list
        Player.random.shuffle(ship)
         
        for ship_number in ship:
            control = True
            
            while control:
                # randomly select a point on the grid
                ship_start_position = self.random_position()

                if self.vacant(ship_start_position, ship_number, grid):
                    for row in range(0, ship_number):
                        #print(f'{ship_start_position} ship : {ship_number}')
                        grid[ship_start_position[0] + row][ship_start_position[1]] = ship_number

                    control = False        
                
        self.player_battleground = grid
        return grid

    def take_turn(self, history):
        '''

        :param history: records the game
                        list that contains one entry per turn
                        dict with 3 keys
                            - 'shots' : list of shots made in the turn
                            - 'hits' : an integer number of hits that your shots made
                            - 'incoming' : opponents list of shots, [] if moved

        :return: a list of tuple/s describing the shots or changed location
                depending on the strategy
        '''
        last_shot = []
        if self.counter > 0:
            last_shot = history[-1]['incoming']
    
        self.player_battleground = self.update_opponent_shots(self.player_battleground, last_shot)
        
        # counting the number of ships left
        self.ships_dd = Player.defaultdict(int) #reset count everytime
        #ships_dd = Player.defaultdict(int)

        for row in self.player_battleground:
            for point in row:
                if point > 0:
                    self.ships_dd[point] += point
                
        self.status = self.ship_status(self.ships_dd)
        
        # Count the number of ships remaining
        number_of_ships = len(self.status)
        
        # Take random shots
        if self.counter > 0:
            shots_list = [number for number in self.random_selection(self.potential_list, number_of_ships)]
        else:
            shots_list = [number for number in self.random_selection(self.potential_list, 5)]      
        
        
        self.counter += 1
        
        return shots_list

    def get_name(self):
        '''

        :return: string - name of the Player
        '''
        return "Syndicate_10"
    
    # function to select a point on the grid
    def random_position(self):
        return [Player.random.randint(0,9), Player.random.randint(0,9)]
    
    # function to check if the position is vacant
    def vacant(self, pos, ship, grid):
        # checking if the ship will fit in the board
        if pos[0] + ship <= 10:
            for i in range(0, ship):
                # checking if any other ship is already present
                if grid[pos[0] + i][pos[1]] != 0:
                    return False
            return True
        return False 
    
    def random_shots(self, num = 5):
        for x in range(0,num):
            yield Player.random.randint(0,9), Player.random.randint(0,9)
            
    def update_opponent_shots(self, grid, incoming_shots):
        for x, y in incoming_shots:
            grid[x][y] = 9
        return grid
    
    def ship_status(self, ships_dd):
        status = []
        for ship in ships_dd.keys():
            # as long as the default dictionary has a key, the ship is breathing
            if ship in [1,2,3,4,5]:
                status.append((ship, ships_dd[ship]/(ship**2)))
        return status
                
    def random_selection(self, potential_list, num = 0):
        if num > len(self.potential_list):
            # If the number of shots remaining are less than the actual ships
            num = len(self.potential_list)

        for x in range(0, num):
            shot = Player.random.choice(self.potential_list)
            self.potential_list.remove(shot)

            yield shot

In [12]:
p1 = Player()

In [13]:
p1.get_name()

'Syndicate_10'

In [14]:
p1.setup_ships()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 4, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 4, 0, 3, 0, 0, 0, 0],
 [5, 0, 2, 4, 0, 3, 0, 0, 0, 0],
 [5, 0, 2, 4, 0, 3, 0, 0, 0, 0],
 [5, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [5, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [5, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [15]:
p1.take_turn(history)

[(7, 2), (5, 5), (4, 4), (5, 1), (7, 0)]

In [9]:
p1.take_turn(history)

[(4, 0), (1, 7), (1, 4), (6, 8), (8, 6)]

In [45]:
len(p1.potential_list)

90

In [46]:
p1.player_battleground

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 9, 0, 0, 0, 0, 0, 0],
 [0, 0, 9, 0, 9, 3, 0, 4, 0, 0],
 [2, 0, 9, 0, 0, 9, 0, 4, 0, 0],
 [2, 0, 0, 0, 5, 3, 0, 4, 0, 0],
 [0, 0, 0, 0, 5, 0, 0, 4, 0, 0],
 [0, 0, 0, 0, 5, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 5, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 5, 0, 0, 0, 0, 0]]

In [42]:
p1.status

[(3, 1.0), (4, 1.0), (2, 1.0), (5, 1.0), (1, 1.0)]

In [16]:
p1.ships_dd

defaultdict(int, {4: 16, 3: 9, 5: 25, 2: 4, 1: 1})

In [17]:
len(p1.ships_dd)

5

In [8]:
history = [{'shots': [(1, 3), (6, 3), (5, 4), (9, 2), (9, 8)],
  'hits': 0,
  'incoming': [(4, 5), (3, 4), (2, 3), (3, 2), (4, 2)]}]

In [38]:
history

[{'shots': [(1, 3), (6, 3), (5, 4), (9, 2), (9, 8)],
  'hits': 0,
  'incoming': [(4, 5), (3, 4), (2, 3), (3, 2), (4, 2)]}]

In [9]:
history[-1]['incoming']

[(4, 5), (3, 4), (2, 2), (6, 9), (2, 9)]

In [6]:
def setup_ships():
        grid = [[0 for i in range(10)] for j in range(10)]
        for ship in range(1, 6):
            for row in range(0, ship):
                grid[row][ship] = ship
        return grid

In [12]:
# function for test purposes only
def reset_grid():
    grid = [[0 for i in range(10)] for j in range(10)]
    return grid

In [11]:
grid = [[0 for i in range(10)] for j in range(10)]
grid

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [14]:
grid[0][3] = 1

In [7]:
setup_ships()

[[0, 1, 2, 3, 4, 5, 0, 0, 0, 0],
 [0, 0, 2, 3, 4, 5, 0, 0, 0, 0],
 [0, 0, 0, 3, 4, 5, 0, 0, 0, 0],
 [0, 0, 0, 0, 4, 5, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 5, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [25]:
random.randint(0,9)

7

In [26]:
# ship numbers
ship = [i for i in range(1,6)]

In [27]:
ship

[1, 2, 3, 4, 5]

In [29]:
# pick a ship at random
random.choice(ship)

3

In [31]:
grid[random.randint(0,9)][random.randint(0,9)] = random.choice(ship)

In [33]:
grid = reset_grid()

In [34]:
# function to select a point on the grid
def random_position():
    return [random.randint(0,9), random.randint(0,9)]

In [35]:
random_position()

[7, 5]

In [52]:
# function to check if the position is vacant
def vacant(pos, ship, grid):
    print(f'position: {pos}, ship: {ship}')
    if pos[0] + ship <= 10:
        for i in range(0, ship):
            if grid[i][pos[1]] != 0:
                return False
        return True
    return False         

In [57]:
vacant(random_position(), random.choice(ship), grid)

position: [0, 6], ship: 3


True

In [49]:
grid[7][0] = '*'

In [51]:
grid = reset_grid()

In [96]:
# ship numbers
ship = [i for i in range(1,6)]

# randomly shuffle the ship list
random.shuffle(ship)

for ship_number in ship:
    
    control = True
    
    # randomly select a ship to place
    #ship_number = random.choice(ship)
    
    # create a list for direction
    direction = [1, 2, 3, 4] # down, up, left, right
    
    #if random.choice(direction) == 1:
    
    print(ship_number)
    
    while control:
        # randomly select a point on the grid
        ship_start_position = random_position()
        
        if vacant(ship_start_position, ship_number, grid):
            for row in range(0, ship_number):
                    grid[ship_start_position[0] + row][ship_start_position[1]] = ship_number
                    
            control = False
        else:   
            print(f'position {ship_start_position} falied, trying again...')
    

4
position: [9, 1], ship: 4
position [9, 1] falied, trying again...
position: [8, 3], ship: 4
position [8, 3] falied, trying again...
position: [3, 4], ship: 4
1
position: [4, 2], ship: 1
5
position: [4, 9], ship: 5
3
position: [9, 4], ship: 3
position [9, 4] falied, trying again...
position: [6, 4], ship: 3
2
position: [0, 2], ship: 2


In [95]:
grid = reset_grid()

In [97]:
grid

[[0, 0, 2, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 2, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 4, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 4, 0, 0, 0, 0, 5],
 [0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
 [0, 0, 0, 0, 3, 0, 0, 0, 0, 5],
 [0, 0, 0, 0, 3, 0, 0, 0, 0, 5],
 [0, 0, 0, 0, 3, 0, 0, 0, 0, 5],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:
for ship in range(1, 6):
            for row in range(0, ship):
                grid[row][ship] = ship

In [5]:
with open('test.py', 'r') as f:
    data = f.read()

In [7]:
type(data)

str

In [7]:
player_battlefield = p1.setup_ships()

In [8]:
player_battlefield

[[0, 0, 2, 0, 5, 0, 0, 0, 0, 0],
 [0, 0, 2, 0, 5, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 5, 1, 0, 0, 0, 0],
 [0, 0, 3, 0, 5, 0, 0, 0, 0, 0],
 [0, 0, 3, 0, 5, 0, 0, 0, 0, 0],
 [0, 0, 3, 0, 0, 0, 0, 4, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 4, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 4, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 4, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [9]:
grid = [[0 for i in range(10)] for j in range(10)]

In [48]:
# Generating the history randomly
mydict = {'shots':[], 'hits':0, 'incoming':[]}
history_list = []

In [49]:
def generate_history(history_list = [], reset = False):
    
    if reset:
        return []
    else:

        shots_list = [number for number in random_shots()]
        hits = random.randint(0,5)
        incoming_list = [number for number in random_shots()]

        mydict['shots'] = shots_list
        mydict['hits'] = hits
        mydict['incoming'] = incoming_list

        return mydict

In [50]:
mydict

{'shots': [], 'hits': 0, 'incoming': []}

In [51]:
def random_shots(num = 5):
    for x in range(0,num):
        yield random.randint(0,9), random.randint(0,9)

In [52]:
for number in random_shots(3):
    print(number)

(3, 9)
(6, 5)
(1, 4)


In [53]:
shots_list = [number for number in random_shots()]
shots_list

[(4, 1), (1, 2), (1, 3), (4, 3), (7, 7)]

In [54]:
def hist_generator(num = 5):
    for x in range(0,num):
        yield generate_history()

In [55]:
history = []
for obj in hist_generator():
    print(obj)
    temp = obj.copy()
    history += [temp]

{'shots': [(7, 3), (5, 2), (9, 4), (6, 1), (2, 6)], 'hits': 3, 'incoming': [(9, 1), (0, 4), (8, 3), (7, 7), (0, 9)]}
{'shots': [(4, 9), (6, 6), (9, 6), (4, 8), (2, 2)], 'hits': 4, 'incoming': [(5, 0), (1, 7), (4, 1), (1, 4), (1, 1)]}
{'shots': [(9, 8), (6, 6), (4, 8), (4, 6), (7, 1)], 'hits': 5, 'incoming': [(0, 5), (7, 0), (2, 2), (8, 7), (0, 6)]}
{'shots': [(6, 3), (3, 7), (3, 2), (2, 0), (5, 4)], 'hits': 5, 'incoming': [(9, 1), (2, 7), (0, 1), (0, 1), (6, 0)]}
{'shots': [(4, 4), (6, 4), (4, 6), (6, 1), (8, 1)], 'hits': 3, 'incoming': [(2, 9), (2, 9), (6, 9), (1, 4), (4, 2)]}


In [18]:
history = [{'shots': [(4, 3), (9, 3), (9, 5), (7, 1), (8, 0)],
  'hits': 2,
  'incoming': [(7, 2), (2, 0), (3, 4), (9, 1), (5, 9)]},
 {'shots': [(4, 7), (7, 1), (3, 7), (4, 9), (9, 2)],
  'hits': 4,
  'incoming': [(8, 0), (0, 0), (6, 5), (1, 1), (3, 5)]},
 {'shots': [(4, 8), (3, 2), (2, 7), (1, 4), (6, 2)],
  'hits': 3,
  'incoming': [(2, 3), (8, 2), (6, 3), (6, 4), (8, 2)]},
 {'shots': [(0, 0), (1, 9), (6, 9), (9, 3), (0, 8)],
  'hits': 1,
  'incoming': [(4, 1), (1, 6), (6, 3), (5, 6), (1, 2)]},
 {'shots': [(1, 3), (6, 3), (5, 4), (9, 2), (9, 8)],
  'hits': 0,
  'incoming': [(8, 9), (3, 4), (2, 2), (6, 9), (2, 9)]}]

In [11]:
history

[{'shots': [(4, 3), (9, 3), (9, 5), (7, 1), (8, 0)],
  'hits': 2,
  'incoming': [(7, 2), (2, 0), (3, 4), (9, 1), (5, 9)]},
 {'shots': [(4, 7), (7, 1), (3, 7), (4, 9), (9, 2)],
  'hits': 4,
  'incoming': [(8, 0), (0, 0), (6, 5), (1, 1), (3, 5)]},
 {'shots': [(4, 8), (3, 2), (2, 7), (1, 4), (6, 2)],
  'hits': 3,
  'incoming': [(2, 3), (8, 2), (6, 3), (6, 4), (8, 2)]},
 {'shots': [(0, 0), (1, 9), (6, 9), (9, 3), (0, 8)],
  'hits': 1,
  'incoming': [(4, 1), (1, 6), (6, 3), (5, 6), (1, 2)]},
 {'shots': [(1, 3), (6, 3), (5, 4), (9, 2), (9, 8)],
  'hits': 0,
  'incoming': [(8, 9), (3, 4), (2, 2), (6, 9), (2, 9)]}]

In [12]:
history[-1]['incoming']

[(8, 9), (3, 4), (2, 2), (6, 9), (2, 9)]

Updating the oppenent grid

In [31]:
grid = [[0 for i in range(10)] for j in range(10)]

In [21]:
for i in history:
    print(i['incoming'])

[(7, 2), (2, 0), (3, 4), (9, 1), (5, 9)]
[(8, 0), (0, 0), (6, 5), (1, 1), (3, 5)]
[(2, 3), (8, 2), (6, 3), (6, 4), (8, 2)]
[(4, 1), (1, 6), (6, 3), (5, 6), (1, 2)]
[(8, 9), (3, 4), (2, 2), (6, 9), (2, 9)]


In [22]:
# latest shot from history
history[-1]['incoming']

[(8, 9), (3, 4), (2, 2), (6, 9), (2, 9)]

In [23]:
last_shot = history[-1]['incoming']

In [28]:
def update_opponent_shots(grid, incoming_shots):
    for x, y in incoming_shots:
        grid[x][y] = 9
    return grid

In [32]:
update_opponent_shots(grid, last_shot)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 9, 0, 0, 0, 0, 0, 0, 9],
 [0, 0, 0, 0, 9, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [30]:
update_opponent_shots(grid, history[-2]['incoming'])

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 9, 0, 0, 0, 9, 0, 0, 0],
 [0, 0, 9, 0, 0, 0, 0, 0, 0, 9],
 [0, 0, 0, 0, 9, 0, 0, 0, 0, 0],
 [0, 9, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 9, 0, 0, 0],
 [0, 0, 0, 9, 0, 0, 0, 0, 0, 9],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [27]:
# player_battlefield = p1.setup_ships()
player_battlefield

[[0, 0, 2, 0, 5, 0, 0, 0, 0, 0],
 [0, 0, 2, 0, 5, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 5, 1, 0, 0, 0, 0],
 [0, 0, 3, 0, 5, 0, 0, 0, 0, 0],
 [0, 0, 3, 0, 5, 0, 0, 0, 0, 0],
 [0, 0, 3, 0, 0, 0, 0, 4, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 4, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 4, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 4, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [33]:
# on the players battlefield
update_opponent_shots(player_battlefield, last_shot)

[[0, 0, 2, 0, 5, 0, 0, 0, 0, 0],
 [0, 0, 2, 0, 5, 0, 0, 0, 0, 0],
 [0, 0, 9, 0, 5, 1, 0, 0, 0, 9],
 [0, 0, 3, 0, 9, 0, 0, 0, 0, 0],
 [0, 0, 3, 0, 5, 0, 0, 0, 0, 0],
 [0, 0, 3, 0, 0, 0, 0, 4, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 4, 0, 9],
 [0, 0, 0, 0, 0, 0, 0, 4, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 4, 0, 9],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [35]:
player_battlefield = update_opponent_shots(player_battlefield, last_shot)

In [34]:
from collections import defaultdict

In [37]:
# counting the number of ships left
ships_dd = defaultdict(int)

for row in player_battlefield:
    for point in row:
        ships_dd[point] += point

In [38]:
ships_dd

defaultdict(int, {0: 0, 2: 4, 5: 20, 9: 45, 1: 1, 3: 9, 4: 16})

In [39]:
player_battlefield

[[0, 0, 2, 0, 5, 0, 0, 0, 0, 0],
 [0, 0, 2, 0, 5, 0, 0, 0, 0, 0],
 [0, 0, 9, 0, 5, 1, 0, 0, 0, 9],
 [0, 0, 3, 0, 9, 0, 0, 0, 0, 0],
 [0, 0, 3, 0, 5, 0, 0, 0, 0, 0],
 [0, 0, 3, 0, 0, 0, 0, 4, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 4, 0, 9],
 [0, 0, 0, 0, 0, 0, 0, 4, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 4, 0, 9],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [40]:
def ship_status(ships_dd):
    status = []
    for ship in ships_dd.keys():
        # as long as the default dictionary has a key, the ship is breathing
        if ship in [1,2,3,4,5]:
            status.append((ship, ships_dd[ship]/(ship**2)))
            
    return status

In [42]:
status = ship_status(ships_dd)

In [43]:
status

[(2, 1.0), (5, 0.8), (1, 1.0), (3, 1.0), (4, 1.0)]

In [44]:
number_of_ships = len(status)

In [45]:
number_of_ships

5

In [23]:
[number for number in random_selection(potential_list,5)]

[(2, 6), (6, 3), (0, 5), (8, 0), (3, 6)]

In [15]:
potential_list = [[0 for i in range(10)] for j in range(10)]

In [17]:
potential_list = []
for i in range(10):
    for j in range (10):
        potential_list.append((i,j))
        

In [19]:
random.choice(potential_list)

(3, 4)

In [21]:
def random_selection(potential_list, num = 0):
    for x in range(0,num):
        shot = random.choice(potential_list)
        yield shot

In [35]:
potential_list.remove((2, 6))

In [28]:
potential_list = []
for i in range(10):
    for j in range(10):
        potential_list.append((i, j))

checkerboard strategy

In [25]:
checkerboard = []
for i in range(1,10,2):
    for j in range(0,10,2):
        checkerboard.append((i, j))

In [2]:
checkerboard

[(1, 0),
 (1, 2),
 (1, 4),
 (1, 6),
 (1, 8),
 (3, 0),
 (3, 2),
 (3, 4),
 (3, 6),
 (3, 8),
 (5, 0),
 (5, 2),
 (5, 4),
 (5, 6),
 (5, 8),
 (7, 0),
 (7, 2),
 (7, 4),
 (7, 6),
 (7, 8),
 (9, 0),
 (9, 2),
 (9, 4),
 (9, 6),
 (9, 8)]

In [17]:
def random_selection(potential_list, checkerboard, num = 5):
    for x in range(0,num):
        shot = random.choice(checkerboard)
        potential_list.remove(shot)
        checkerboard.remove(shot)
        yield shot   

In [8]:
len(potential_list)

100

In [9]:
len(checkerboard)

25

In [11]:
shots_list = [number for number in random_selection(potential_list, checkerboard, 5)]

In [12]:
shots_list

[(7, 8), (9, 4), (1, 2), (7, 6), (3, 8)]

In [13]:
len(potential_list)

95

In [14]:
len(checkerboard)

20

In [26]:
[number for number in random_selection(potential_list, checkerboard, 5)]

[(7, 6), (3, 4), (3, 2), (7, 0), (1, 8)]

In [21]:
checkerboard = []

In [27]:
len(potential_list)

95

In [1]:
f = lambda x:x**2

In [2]:
f(3)

9

In [16]:
list(map(f,[i for i in range(10)]))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [24]:
last_shot = history[-1]['incoming'] if counter > 0 else []

In [23]:
counter = 1

In [25]:
last_shot

[(8, 9), (3, 4), (2, 2), (6, 9), (2, 9)]

In [26]:
histories = [[],[]]

In [32]:
if histories[0]:
    print('1')
else:
    print('0')

0


In [1]:
if history:

SyntaxError: unexpected EOF while parsing (<ipython-input-1-e94f74e2d3cc>, line 1)

In [9]:
players = [Player(), Player()]

grids = [p.setup_ships() for p in players]
#grids = map(lambda p: p.setup_ships(), players)